# Project: Investigate a Dataset of No-Show Appointment

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

This dataset collects information from 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment.

## Column Description
<ul>
<li><b>PatientId</b> = Identification of a patient.</li>
<li><b>AppointmentID</b> = Identification of each appointment. </li>
<li><b>Gender</b> = Male or Female.</li>
<li><b>AppointmentDay</b> = The day of the actual appointment, when they have to visit the doctor. </li>
<li><b>ScheduledDay</b> = The day they called to book the appointment.</li>
<li><b>Age</b> = How old is the patient.</li>
<li><b>Neighbourhood</b> = Where the appointment takes place. </li>
<li><b>Scholarship</b> = Ture(1) or False(0).</li>
<li><b>Hypertension</b> = True(1) or False(0).</li>
<li><b>Diabetes</b> = True(1) or False(0).</li>
<li><b>Alcoholism</b> = True(1) or False(0).</li>
<li><b>Handcap</b> = True(1) or False(0).</li>
<li><b>SMS_received</b> = 1 or more messages sent to the patient.</li> 
<li><b>No-show</b> = Yes or No.</li>    
</ul>

### Note:
It is worth to notice that the 'No-Show' column have <b>No</b> if the patient showed up to their appointment, and <b>Yes</b>, if they did not show up.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling


> Pandas **read_csv** method loads the dataset into Pandas **DataFrame** for further analysis. **head** method can be used to print out a few rows of the dataset to check what kind of data we are dealing with.

In [2]:
df = pd.read_csv('noshowappointments.csv')
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


> Changing the column header names to **snake case** naming convention. With this, column names will be consistent with each other and can be use using dot notation. Also, correcting the spelling mistake in some column names.

In [3]:
columns = ['patient_id', 'appointment_id', 'gender', 'scheduled_day', 'appointment_day', 'age', 'neighbourhood', 'scholarship', 'hypertension', 'diabetes', 'alcoholism', 'handicap', 'sms_received', 'no_show']

In [4]:
df.columns = columns

> Pandas **info** method give detail about *Total no. of not null entries* for each and every column.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
patient_id         110527 non-null float64
appointment_id     110527 non-null int64
gender             110527 non-null object
scheduled_day      110527 non-null object
appointment_day    110527 non-null object
age                110527 non-null int64
neighbourhood      110527 non-null object
scholarship        110527 non-null int64
hypertension       110527 non-null int64
diabetes           110527 non-null int64
alcoholism         110527 non-null int64
handicap           110527 non-null int64
sms_received       110527 non-null int64
no_show            110527 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


Every column has same count. There is no column having **null** in it

> Pandas **to_datetime** method is used to convert *scheduled_day* and *appointment_day* from string to datetime object. We are splitting the string on the basis of position of 'T' to get only the date part. With this, we can use them for analysis that involve manipulation of dates.

In [6]:
df.scheduled_day = df.scheduled_day.apply(lambda x : x.split('T')[0])
df.appointment_day = df.appointment_day.apply(lambda x : x.split('T')[0])

In [7]:
df.scheduled_day = pd.to_datetime(df.scheduled_day)
df.appointment_day = pd.to_datetime(df.appointment_day)

> Since *no_show* column is confusing in interpretation. We will create a new column *patient_showed_up*, which is the inverse of *no_show* column.

In [8]:
df['no_show'].value_counts()

No     88208
Yes    22319
Name: no_show, dtype: int64

In [9]:
df['patient_showed_up'] = df['no_show'].apply(lambda x : "No" if x == "Yes" else "Yes")

In [10]:
df['patient_showed_up'].value_counts()

Yes    88208
No     22319
Name: patient_showed_up, dtype: int64

> Dropping the *no_show* column using **Drop** method, since this column is not required any more

In [11]:
df.drop(columns=['no_show'], axis=1, inplace=True)

In [12]:
df.head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hypertension,diabetes,alcoholism,handicap,sms_received,patient_showed_up
0,2.987250e+13,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,Yes
1,5.589978e+14,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,Yes
2,4.262962e+12,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,Yes
3,8.679512e+11,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,Yes
4,8.841186e+12,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,Yes


> Pandas **duplicated** method can be used to check if there are multiple rows having same data.

In [13]:
df.duplicated().value_counts()

False    110527
dtype: int64

Since there is no 'True' entry in the result. There are no duplicate rows in this dataframe.

### Data Cleaning (Replace this with more specific notes!)

> Pandas **query** method is used to check whether there are any rows for which the Age is equal to or less than zero.

In [14]:
df.query('age <= 0').size

49560

This query returns 49560 rows. This means these many rows are having age value of equal to or less than zero. So, we will remove these rows from the dataframe.

> Pandas **drop** method is used to drop all the rows having age value equal to or less than zero.

In [15]:
df.drop(index=df.query('age <= 0').index, inplace=True)

> Droping *appointment_id* columns because they are not much of a use as they contains different values for each row and do not contribute that much in our analysis.

In [16]:
df.drop(columns=['appointment_id'], axis=1, inplace=True)

> Now, after removing *appointment_id*. We will check for duplicate rows. i.e. Patient have different appointment for same day, same time at same hospital.

In [17]:
df.duplicated().value_counts()

False    102856
True       4131
dtype: int64

There are 4131 duplicate rows. To remove the duplicate rows. We will use **drop_duplicates** method

In [18]:
df.drop_duplicates(inplace=True)

In [19]:
df.duplicated().value_counts()

False    102856
dtype: int64

> Pandas **unique** method returns unique value present in a column of a dataframe

The unique method returns 5 values for *handicap*. This might be because they are checking to what extent a person is handicapped. Since we are only checking whether a person is handicap or not, we will change the values other than 0 or 1 to **1**.

In [20]:
df['handicap'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [21]:
handicap_val_index = df.query('handicap !=1 & handicap !=0').index

In [22]:
df.loc[handicap_val_index, 'handicap'] = 1

In [23]:
df['handicap'].unique()

array([0, 1], dtype=int64)

> Getting the time span betwwen the time when the patient book the appointment and the day the appointment is scheduled, and storing it in new column *time_diff*

In [24]:
df['time_diff'] = df['appointment_day'].dt.date - df['scheduled_day'].dt.date

Convert timedelta object to integer number for graphing purposes.

In [25]:
df['time_diff'] = df['time_diff'].apply(lambda td : int(td / np.timedelta64(1, 'D')))

Checking for negative time difference. i.e. appointment day after schuduled day.

In [26]:
df.query('time_diff < 0')['time_diff'].value_counts()

-1    4
-6    1
Name: time_diff, dtype: int64

There are 5 rows having negative value of time difference. We will drop these rows.

In [27]:
df.drop(index=df.query('time_diff < 0').index, inplace=True)

In [28]:
df['time_diff'].max()

179

The **maximum** time difference between schedule day and appointment day is 179 days.

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102851 entries, 0 to 110526
Data columns (total 14 columns):
patient_id           102851 non-null float64
gender               102851 non-null object
scheduled_day        102851 non-null datetime64[ns]
appointment_day      102851 non-null datetime64[ns]
age                  102851 non-null int64
neighbourhood        102851 non-null object
scholarship          102851 non-null int64
hypertension         102851 non-null int64
diabetes             102851 non-null int64
alcoholism           102851 non-null int64
handicap             102851 non-null int64
sms_received         102851 non-null int64
patient_showed_up    102851 non-null object
time_diff            102851 non-null int64
dtypes: datetime64[ns](2), float64(1), int64(8), object(3)
memory usage: 11.8+ MB


In [30]:
df.to_csv('data_sheet.csv', index=False)

<a id='eda'></a>
## Exploratory Data Analysis

### 1.) What is the proportion of patients showed up for their appointment ? 

In [ ]:
plt.figure(figsize=(10,8))
_, arr_bins, _ = plt.hist(df['patient_showed_up'], bins=2, rwidth=.95)
plt.xticks([arr_bins[:2].mean(), arr_bins[1:].mean()], ['No', 'Yes'])
plt.ylabel('No. of show ups')
plt.xlabel('Patient showed up')
plt.title('Histogram of Patients showed up VS. Patients didn\'t show up');

> From the above histogram, It is clear that a large no. of patients did show up for their appointment.

### 2.) What is the distrubution of patients in different age group ?

In [ ]:
plt.figure(figsize=(18,12))
plt.hist(df['age'], bins=50)
plt.axis([0, 100, 0, 6000])
plt.title('Histogram of different age groups of patients')
plt.xlabel('Age')
plt.ylabel('age frequency');

> A great number of patients are under the age of 50. Also highest number of patient are infants.

### 3.) What is the proportion of patients having scholarship ?

In [ ]:
plt.figure(figsize=(10,8))
_, arr_bins,_ = plt.hist(df['scholarship'].values, bins=2, rwidth=.95)
plt.title('Histogram of Patients with scholarship')
plt.xticks([arr_bins[:2].mean(), arr_bins[1:].mean()], ['No', 'Yes'])
plt.ylabel('No. of scholarships')
plt.xlabel('Scholarship')
plt.xlabel('Patient showed up');

> We can assume from the above histogram that a large number of patients do not have scholarship (did not enroll for scholarship program)

### 4.) For patients having a scholarship or not, What number of patients showed up for their appointment ?

In [ ]:
scholarship_unstacked = df.groupby('scholarship')['patient_showed_up'].value_counts().unstack()

In [ ]:
plt.figure(figsize=(8,5))

plt.bar(x=range(len(scholarship_unstacked.index)), height=scholarship_unstacked['Yes'], alpha=0.5, label='Yes')
plt.bar(x=range(len(scholarship_unstacked.index)), height=scholarship_unstacked['No'], alpha=0.5, color='red', label='No')

plt.xticks(range(len(scholarship_unstacked.index)), ('Patients with No Schoparship', 'Patients with Schoparship'))

plt.xlabel('Patient has scholarship')
plt.ylabel('No. of patients')

plt.title('Patient with/without scholarship showed for hospital appointment')
plt.legend(title='Patient showed up');

> The no. of people having scholarship is way less than poeple with no schoparship. People with no scholarship are visiting the hospital more than people who has scholarship.

### 5.) How many patients having certain conditions showed up for their appointment ?

In [ ]:
patients_with_conditions = df.query('patient_showed_up == "Yes"').groupby(['hypertension', 'diabetes', 'alcoholism'])['patient_showed_up'].value_counts()

In [ ]:
patients_with_conditions

In [ ]:
patients_with_conditions.unstack().plot(kind='bar', figsize=(15,8))
plt.yticks(np.linspace(0, patients_with_conditions.max(), 20))
plt.xticks(range(8), ['No Condition', 'Alcoholism', 'Diabetes', 'Diabetes, \nAlcoholism', 'Hypertension', 
                      'Hypertension, \nAlcoholism', 'Hypertension, \nDiabetes', 'Hypertension, \nDiabetes, Alcoholism'], 
          rotation=0)
plt.title('People with different conditions showed up for appointment')
plt.xlabel('Different Conditions: Hypertension, Alcoholism, Diabetes')
plt.ylabel('No. of patients')
plt.legend(['Yes'], title='Patient showed up');

> Apart from people with no condition, Patients with Hypertension are more likely to visit the hospital than people with any other conditions.

### 6.) What is the number number of patients who received SMS and who don't received showed up for their appointment ?

In [ ]:
sms_received_grouped = df.groupby('sms_received')['patient_showed_up'].value_counts()

In [ ]:
sms_received_unstack = sms_received_grouped.unstack()

In [ ]:
plt.figure(figsize=(8,5))

plt.bar(x=range(len(sms_received_unstack.index)), height=sms_received_unstack['Yes'], alpha=0.5, label='Yes')
plt.bar(x=range(len(sms_received_unstack.index)), height=sms_received_unstack['No'], alpha=0.5, color='red', label='No')

plt.xticks(range(len(sms_received_unstack.index)), ('Patients who didn\'t receive SMS', 'Patients who received SMS'))

plt.ylabel('No. of patients')
plt.title('Patient who received SMS VS. Patient who didn\'t receive SMS')
plt.xlabel('Patient received or didn\'t receive SMS')
plt.legend(title='Patient showed up');

> The no. of patients who didn't receive the SMS is much higher than those who received SMS. Still a large proportion of those patients went for their appointment.

### 7.) What is the proportion of male and female patients who showed up or didn't show up for their appointment ?

In [ ]:
gender_unstacked = df.groupby('gender')['patient_showed_up'].value_counts().unstack()

In [ ]:
gender_unstacked.plot(kind='bar', title='Female VS Male who showed up for appointment', figsize=(8,5))

plt.xlabel('Female and Male Patient')
plt.ylabel('No. of patients')

plt.xticks(range(len(gender_unstacked.index)), ['Female', 'Male'], rotation=0);
plt.legend(['No', 'Yes'], title='Patient showed up')

> Females are more likely to go to hospital as compare to male patients. They are more concern about their health and fitness than male patients.

### 8.) Which Hospital is having maximum no of show ups ? And which hospital is having maximum no of no show ups ?

In [ ]:
hospitals_in_neighbourhood = df.groupby('neighbourhood')['patient_showed_up'].value_counts().unstack()

In [ ]:
plt.figure(figsize=(20,15))

plt.subplot(2, 1, 1)
plt.bar(x=range(len(hospitals_in_neighbourhood.index)), height=hospitals_in_neighbourhood['No'], label='No')
plt.xticks(range(len(hospitals_in_neighbourhood.index)), hospitals_in_neighbourhood.index, rotation=90)
plt.ylabel('No. of Patients')
plt.xlabel('Hospital in Neighbourhood')
plt.title('Hospital with no show count of patients')
plt.legend(title='Patient showed up');

plt.subplots_adjust(hspace=1.2)

plt.subplot(2, 1, 2)
plt.bar(x=range(len(hospitals_in_neighbourhood.index)), height=hospitals_in_neighbourhood['Yes'], color='orange', label='Yes')
plt.xticks(range(len(hospitals_in_neighbourhood.index)), hospitals_in_neighbourhood.index, rotation=90)
plt.ylabel('No. of Patients')
plt.xlabel('Hospital in Neighbourhood')
plt.title('Hospital with show count of patients')
plt.legend(title='Patient showed up');

> *'Jardim Camburi'* Hospital has maximum no. of visits from patients who booked an appointment. Also same hospital has maximum number of no show up.

### 9.) Does much later appointment date after schedule day became the reason of no show ?

In [ ]:
diff_in_time_grp = df.groupby('time_diff')['patient_showed_up'].value_counts().unstack()

In [ ]:
plt.figure(figsize=(15,12))
plt.bar(x=range(len(diff_in_time_grp.index)), height=diff_in_time_grp['No'])

plt.subplot(2, 1, 1)
plt.bar(x=range(len(diff_in_time_grp.index)), height=diff_in_time_grp['No'], label='No')
plt.xticks(range(0, len(diff_in_time_grp.index), 5), rotation=0)
plt.ylabel('No. of Patients')
plt.xlabel('Time difference in days')
plt.title('Relation between \'Difference of schedule day apointment day\' & \'Patient show up behavior\'')
plt.legend(title='Patient showed up', loc="upper right")

plt.subplots_adjust(hspace=.5)

plt.subplot(2, 1, 2)
plt.bar(x=range(len(diff_in_time_grp.index[1:])), height=diff_in_time_grp['Yes'][1:], color='orange', label='Yes')
plt.xticks(range(0, len(diff_in_time_grp['Yes'].index), 5), rotation=0)
plt.ylabel('No. of Patients')
plt.xlabel('Time difference in days')
plt.title('Relation between \'Difference of schedule day apointment day\' & \'Patient show up behavior\'')
plt.legend(title='Patient showed up', loc="upper right");

> People are more likely to show up for their appointment if the time span between schedule day and appointment day is small.

<a id='conclusions'></a>
## Conclusions

### From all the figures and graphs we can conclude that:

<ul>
    <li> Most of the patients showed up for their appointment.</li>
    <li> Most of the patients are under the age of 50. The highest number of patient are infants. </li>
    <li> A large number of patients do not have scholarship or they did not enroll for scholarship program. </li>
    <li> There are very less people having scholarship. People with no scholarship are visiting the hospital more than people having scholarship. </li>
    <li> There are a lot of patients who didn't received notification SMS for their appointment. Still they went for their appointment.</li>
    <li> Females are more likely to go to hospital as compare to male patients.</li>
    <li> Except from *'No Condition'*, Patients having Hypertension are visiting hospitals for their appointment.</li>
    <li> *'Jardim Camburi'* Hospital has maximum show ups and maximum no show ups.</li>
    <li> People are more likely to show up for their appointment if the time span between schedule day and appointment day is small.</li>
</ul>

**Limitations:** Since we didn't use any statistical tests. The results might not be fully correct. The analysis is done just to get some basic insights from the data.